# Business Understanding

# Data Understanding

In [25]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [26]:
df = pd.read_csv('data/Aviation_Data.csv', low_memory=False)
#keep in mind what the dataframe means. these are accidents.
#go through what columns to keep and rows by null values. might be ~50,000 values.

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90348 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      90348 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50249 non-null  object 
 9   Airport.Name            52790 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87572 non-null  object 
 14  Make                    88826 non-null

In [28]:
df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


## Data Preparation

I will only be accounting for airline make and models that are in the United States, so I will be removing any rows that are not the United States. Because I am measuring off of total fatal injuries, total serious injuries, total minor injuries, total uninjured, and injury severity, I will remove unnecessary columns. In addition to this, I'll keep some conditions to track safety. These safety conditions to keep in mind are number of engines, engine type, amateur built, weather condition, and flight level.

The reason I am only viewing the United States crashes is because there is low cardinality and low uniform data. Taking into account only US accidents will still give me a chance to analyze majority of the data in our dataframe.

In [19]:
#First I want to examine all columns in the data
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [55]:
#Creating a dataframe that only shows accidents in the United States
df_us = df[df['Country'] == 'United States']
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82248 entries, 0 to 90347
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                82248 non-null  object 
 1   Investigation.Type      82248 non-null  object 
 2   Accident.Number         82248 non-null  object 
 3   Event.Date              82248 non-null  object 
 4   Location                82237 non-null  object 
 5   Country                 82248 non-null  object 
 6   Latitude                32265 non-null  object 
 7   Longitude               32255 non-null  object 
 8   Airport.Code            49189 non-null  object 
 9   Airport.Name            51654 non-null  object 
 10  Injury.Severity         82140 non-null  object 
 11  Aircraft.damage         80269 non-null  object 
 12  Aircraft.Category       28154 non-null  object 
 13  Registration.Number     82197 non-null  object 
 14  Make                    82227 non-null

In [56]:
#I will now remove unwanted columns
#I am removing Country because these are all already United States accidents
cols_to_remove = ['Investigation.Type','Accident.Number','Event.Date','Location','Country',
                  'Aircraft.damage','Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 
                  'Registration.Number', 'FAR.Description', 'Report.Status', 'Publication.Date']
df_us = df_us.drop(columns = cols_to_remove)
df_us.head()

,Event.Id,Injury.Severity,Aircraft.Category,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,Schedule,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight
0,20001218X45444,Fatal(2),NaN,Stinson,108-3,No,1.0,Reciprocating,NaN,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise
1,20001218X45447,Fatal(4),NaN,Piper,PA24-180,No,1.0,Reciprocating,NaN,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown
2,20061025X01555,Fatal(3),NaN,Cessna,172M,No,1.0,Reciprocating,NaN,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise
3,20001218X45448,Fatal(2),NaN,Rockwell,112,No,1.0,Reciprocating,NaN,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise
4,20041105X01764,Fatal(1),NaN,Cessna,501,No,NaN,NaN,NaN,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach


In [57]:
#Because our business problem is which aircraft is lowest risk, I am curious about looking at the value counts
df_us['Aircraft.Category'].value_counts()

Airplane             24229
Helicopter            2723
Glider                 503
Balloon                229
Gyrocraft              172
Weight-Shift           161
Powered Parachute       90
Ultralight              25
WSFT                     9
Unknown                  4
Blimp                    4
Powered-Lift             3
ULTR                     1
Rocket                   1
Name: Aircraft.Category, dtype: int64

In [61]:
#Here I want to figure out how to handle my NaN values for Aircraft.Category
df_us['Aircraft.Category'].isna().sum()

54094

In [62]:
#There a lot of missing aircraft categories that are NaNs. 
#Because these NaN values are associated to Airplane Makes, I'll convert these values to 'Airplane'
df_us['Aircraft.Category'] = df_us['Aircraft.Category'].fillna('Airplane')
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82248 entries, 0 to 90347
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                82248 non-null  object 
 1   Injury.Severity         82140 non-null  object 
 2   Aircraft.Category       82248 non-null  object 
 3   Make                    82227 non-null  object 
 4   Model                   82210 non-null  object 
 5   Amateur.Built           82227 non-null  object 
 6   Number.of.Engines       80373 non-null  float64
 7   Engine.Type             79225 non-null  object 
 8   Schedule                10297 non-null  object 
 9   Purpose.of.flight       79819 non-null  object 
 10  Air.carrier             14495 non-null  object 
 11  Total.Fatal.Injuries    71594 non-null  float64
 12  Total.Serious.Injuries  70873 non-null  float64
 13  Total.Minor.Injuries    71519 non-null  float64
 14  Total.Uninjured         77243 non-null

In [63]:
#The next thing I would want to analyze is how to manage the NaN values for the columns I'm evaluating.
#For this exercise, I will assume the NaN values can just be 0.0, in this context/data NaN would mean no injury.
#If there is a NaN value in any of my injury columns, it means there are no injuries.
#Although there are other columns with NaN values, I will only worry for now about my injury columns.

df_us['Total.Fatal.Injuries'] = df_us['Total.Fatal.Injuries'].fillna(0.0)
df_us['Total.Serious.Injuries'] = df_us['Total.Serious.Injuries'].fillna(0.0)
df_us['Total.Minor.Injuries'] = df_us['Total.Minor.Injuries'].fillna(0.0)
df_us['Total.Uninjured'] = df_us['Total.Uninjured'].fillna(0.0)

In [64]:
#Verify it worked and filled NaN with 0.0
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82248 entries, 0 to 90347
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                82248 non-null  object 
 1   Injury.Severity         82140 non-null  object 
 2   Aircraft.Category       82248 non-null  object 
 3   Make                    82227 non-null  object 
 4   Model                   82210 non-null  object 
 5   Amateur.Built           82227 non-null  object 
 6   Number.of.Engines       80373 non-null  float64
 7   Engine.Type             79225 non-null  object 
 8   Schedule                10297 non-null  object 
 9   Purpose.of.flight       79819 non-null  object 
 10  Air.carrier             14495 non-null  object 
 11  Total.Fatal.Injuries    82248 non-null  float64
 12  Total.Serious.Injuries  82248 non-null  float64
 13  Total.Minor.Injuries    82248 non-null  float64
 14  Total.Uninjured         82248 non-null

In [ ]:
#Now I would like to make my first visual.
#I'm curious about injury severity to aircraft category

# Exploratory Data Analysis

## Conclusions

## Limitations

## Recommendations

## Next Steps